# Criação de dados

A proposta deste notebook é criar um dataframe de dados mockados contendo as seguintes colunas:
- id
- data
- usuário
- empresa

In [10]:
import pandas as pd
from faker import Faker
import uuid
from datetime import datetime
from random import choice

### Imports
- **pandas**: para criar uma tabela
- **faker**: para criar os dados
- **uuid**: para chaves primárias unicas
- **datetime e random**: complementares

In [11]:
# Instanciando o Faker
fake = Faker(["pt_BR"])

In [12]:
# Quantidade de registros na tabela

lines = 500_000

In [13]:
# Criação de dados 

base_company_list = [fake.unique.company_id() for _ in range(int(1_000))] # 1k emrpesas
base_user_list = [fake.unique.email() for _ in range(int(20_000))] # 20k de usuários

# para um contexto mais lógico, onde um usuário sempre pertencer à uma mesma empresa
user_company = [(base_user, choice(base_company_list)) for base_user in base_user_list]
full_user_company = [choice(user_company) for _ in range(lines)]
 
# Criação dos dados
data = {
    "id": [uuid.uuid4() for _ in range(lines)],
    "insert_date": [fake.date_time_between(start_date=datetime(2023, 10, 1)) for _ in range(lines)],
    "username": [user[0] for user in full_user_company],
    "company": [company[1] for company in full_user_company],
}


In [14]:
# Criação do dataframe
df = pd.DataFrame(data)

In [15]:
# Apenas vendo como ficou a tabela
df.head()

,id,insert_date,username,company
0,d64a9a47-5c2f-4194-a8db-39407582398e,2023-10-08 22:15:20,andregoncalves@example.org,72389450000129
1,e94dd7f1-1000-4afa-aad1-1956d46d4e8b,2023-11-04 11:17:39,manuela26@example.net,59120834000183
2,04e9b5f0-d574-41ec-bf53-dfdcfd4b8b5b,2023-11-07 06:54:08,rvieira@example.com,10697432000157
3,6bbb5e55-d775-4fa9-8c17-5617f5dbb2ec,2023-11-06 16:00:05,sophiamartins@example.com,69275348000198
4,3dae1581-8907-4747-a31a-7d805fa1055f,2023-10-12 14:11:14,freitasana-sophia@example.org,85213407000150


In [16]:
# Vendo os tipos das colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id           500000 non-null  object        
 1   insert_date  500000 non-null  datetime64[ns]
 2   username     500000 non-null  object        
 3   company      500000 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 15.3+ MB


In [17]:
# Gerando o arquivo SQL

# Este foi o filename que usei ao rodar no meu pc para criar o script
# filename = "../docker_databases/init_src.sql"

# Estou deixando este apenas para que possa executar no seu computador sem comprometer o resultado
# das analises que realizei.
filename = "../docker_databases/init_avaliacao.sql"

with open(filename, "w") as f:
    f.write("-- Criação do banco de dados\n")
    f.write("CREATE DATABASE sourcedb;\n")
    f.write("\\c sourcedb;\n")
    f.write("\n\n-- Criação da tabela")
    f.write("""
    CREATE TABLE t_ponto (
          id VARCHAR(255) PRIMARY KEY,
          insert_date TIMESTAMP,
          username VARCHAR(255),
          company VARCHAR(255)
    );\n
    """)
    
    f.write("\n\n-- Exportação dos dados\n\n")
    for i in df.index:
        f.write(f"INSERT INTO t_ponto (id, insert_date, username, company) VALUES ('{str(df.iloc[i, 0])}', '{df.iloc[i, 1]}', '{df.iloc[i, 2]}', '{df.iloc[i, 3]}');\n")

In [18]:
print("Fim")

Fim


## Conclusão

foi gerado um script sql para que possa ser instanciado o container com o banco ja contendo as informações. Sempre que o container for executado no composer, ele ja irá conter os dados.